# Import packages

In [1]:
import os

import h5py
import sys
import numpy as np
import pandas as pd
import copy
import tqdm
import matplotlib.pyplot as plt

import ast
sys.path.append(r"C:\Users\shiwei\Documents")

print(os.getpid())

18872


In [2]:
# module to quickly get Gene ID from rna name
from gprofiler import GProfiler

# Define functions

## prepare dict to map Gene ID

In [3]:
_data_folder = r'\\10.245.74.158\Chromatin_NAS_8\MERFISH\v2\20220304-storm6_M1'
codebook_fname = os.path.join(_data_folder, 'codebook_0_M1_codebook.csv')
codebook_df = pd.read_csv(codebook_fname)
MOP_genes = codebook_df['name'].tolist()
#MOP_genes = [g for g in MOP_genes if 'Blank' not in g]

In [4]:
len(MOP_genes)

252

In [5]:
gp = GProfiler(return_dataframe=True)
search_df = gp.convert(organism='mmusculus',
            query=MOP_genes,
            #target='hsapiens'
                      )
gene_id_map_dict = {k:v for k,v in zip(search_df['incoming'],search_df['converted'])}
gene_id_map_dict 

{'1700022I11Rik': 'ENSMUSG00000028451',
 '1810046K07Rik': 'None',
 '5031425F14Rik': 'ENSMUSG00000085129',
 '5730522E02Rik': 'ENSMUSG00000032985',
 'Acta2': 'ENSMUSG00000035783',
 'Adam2': 'ENSMUSG00000022039',
 'Adamts2': 'ENSMUSG00000036545',
 'Adamts4': 'ENSMUSG00000006403',
 'Adra1b': 'ENSMUSG00000050541',
 'Alk': 'ENSMUSG00000055471',
 'Ankfn1': 'ENSMUSG00000047773',
 'Ano4': 'ENSMUSG00000035189',
 'Aqp4': 'ENSMUSG00000024411',
 'Asic4': 'ENSMUSG00000033007',
 'B4galnt2': 'ENSMUSG00000013418',
 'B4galnt3': 'ENSMUSG00000041372',
 'Barx2': 'ENSMUSG00000032033',
 'Bcl11b': 'ENSMUSG00000048251',
 'Bdnf': 'ENSMUSG00000048482',
 'Bgn': 'ENSMUSG00000031375',
 'Blnk': 'ENSMUSG00000061132',
 'Bmpr1b': 'ENSMUSG00000052430',
 'Brinp3': 'ENSMUSG00000035131',
 'C1ql3': 'ENSMUSG00000049630',
 'C1qtnf7': 'ENSMUSG00000061535',
 'Cacng5': 'ENSMUSG00000040373',
 'Calb2': 'ENSMUSG00000003657',
 'Camk2d': 'ENSMUSG00000053819',
 'Car3': 'ENSMUSG00000027559',
 'Cbln2': 'ENSMUSG00000024647',
 'Cbln4': 'E

In [6]:
# manually add the failed genes
gene_id_map_dict.update({'1-Mar':'ENSMUSG00000036469'})
gene_id_map_dict.update({'1810046K07Rik':'ENSMUSG00000036027'})
gene_id_map_dict.update({'Fam19a2':'ENSMUSG00000044071'})
gene_id_map_dict.update({'Fam84b':'ENSMUSG00000072568'})

gene_id_map_dict 

{'1700022I11Rik': 'ENSMUSG00000028451',
 '1810046K07Rik': 'ENSMUSG00000036027',
 '5031425F14Rik': 'ENSMUSG00000085129',
 '5730522E02Rik': 'ENSMUSG00000032985',
 'Acta2': 'ENSMUSG00000035783',
 'Adam2': 'ENSMUSG00000022039',
 'Adamts2': 'ENSMUSG00000036545',
 'Adamts4': 'ENSMUSG00000006403',
 'Adra1b': 'ENSMUSG00000050541',
 'Alk': 'ENSMUSG00000055471',
 'Ankfn1': 'ENSMUSG00000047773',
 'Ano4': 'ENSMUSG00000035189',
 'Aqp4': 'ENSMUSG00000024411',
 'Asic4': 'ENSMUSG00000033007',
 'B4galnt2': 'ENSMUSG00000013418',
 'B4galnt3': 'ENSMUSG00000041372',
 'Barx2': 'ENSMUSG00000032033',
 'Bcl11b': 'ENSMUSG00000048251',
 'Bdnf': 'ENSMUSG00000048482',
 'Bgn': 'ENSMUSG00000031375',
 'Blnk': 'ENSMUSG00000061132',
 'Bmpr1b': 'ENSMUSG00000052430',
 'Brinp3': 'ENSMUSG00000035131',
 'C1ql3': 'ENSMUSG00000049630',
 'C1qtnf7': 'ENSMUSG00000061535',
 'Cacng5': 'ENSMUSG00000040373',
 'Calb2': 'ENSMUSG00000003657',
 'Camk2d': 'ENSMUSG00000053819',
 'Car3': 'ENSMUSG00000027559',
 'Cbln2': 'ENSMUSG00000024647'

## functions to convert data format

In [7]:
## function to convert
def format_4dn_dataframe_RNA (barcode_df, codebook_df, required_cols, gene_id_map_dict, ):
    
    from tqdm import tqdm
    
    # 1. get and rename required column from decoded df
    df_4dn = barcode_df[required_cols]
    df_4dn.index.name = 'Spot_ID'
    rename_dict = {'fov': 'FOV_ID', 
                   'x':'X', 
                   'y':'Y',
                   'global_x':'x_global',
                   'global_y':'y_global',
                   'global_z':'z_global',
                   #'experiment':'RNA_experiment_ID',
                   #'cell_index':'Cell_ID',
                   'barcode_id':'barcode_id'
        
                     }
    df_4dn = df_4dn.rename(columns=rename_dict)
    # insert Z as 0
    df_4dn.insert(2, 'Z', df_4dn['z_global'])
    
    # 2. get RNA name and RNA ID from codebook
    df_4dn['RNA_name'] = df_4dn['barcode_id'].apply(lambda x: codebook_df.loc[x]['name'])
    df_4dn['Gene_ID'] = df_4dn['RNA_name'].apply(lambda x: gene_id_map_dict[x])
    df_4dn['RNA_ID'] = df_4dn['barcode_id'].apply(lambda x: codebook_df.loc[x]['id'])

    # 3. reset index
    df_4dn.reset_index(inplace=True)
    df_4dn=df_4dn[['Spot_ID','X','Y','Z','RNA_name','Gene_ID',
                   'RNA_ID','FOV_ID','x_global','y_global','z_global','barcode_id',]]
    
    return df_4dn




import multiprocessing as mp
from functools import partial
import inspect

# create temp py to parallel
# make a slight change to the py name so it does not interferes with others
def parallel_task(func, zipped_iterables, ):
    import os, time
    cwd = os.getcwd()
    print ("Write in the function to multiprocess as a temp file.")
    
    with open(os.path.join(cwd,f'tmp_func_rna.py'), 'w') as file:
        file.write(inspect.getsource(func).replace(func.__name__, "function_to_mp_rna"))
        
    from tmp_func_rna import function_to_mp_rna

    if __name__ == '__main__':
        start_time = time.time()
        with mp.Pool(16) as mp_pool:
            mp_res = mp_pool.starmap(function_to_mp_rna, zipped_iterables,chunksize=1)
            mp_pool.close()
            mp_pool.join()
            mp_pool.terminate()
        
        elapsed_time = time.time() - start_time
        hours, rem = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print (f"Complete multiprocess; remove the temp file for the function.")
        print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        if os.path.exists(os.path.join(cwd,f'tmp_func_rna.py')):
            os.remove(os.path.join(cwd,f'tmp_func_rna.py'))
        
        return mp_res
    else:
        raise "Not in Jupyter Notebook"


## prepare header lines

In [8]:
table_name_list = ['4dn_FOF-CT_core', '4dn_FOF-CT_cell', '4dn_FOF-CT_rna_spot', '4dn_FOF-CT_demultiplexing']
curr_table_name = '4dn_FOF-CT_rna_spot'
add_table_name = ', '.join([t for t in table_name_list if t!=curr_table_name])
add_table_name

'4dn_FOF-CT_core, 4dn_FOF-CT_cell, 4dn_FOF-CT_demultiplexing'

In [9]:
header_dict_v0 = {'##FOF-CT_version=':'v0.1',
                  '##Table_namespace=':'4dn_FOF-CT_RNA',
                  '#lab_name:':'Xiaowei Zhuang lab', 
                  '#experimenter_name:':'Pu Zheng, Shiwei Liu',
                  '#experimenter_contact:':'pu_zheng@g.harvard.edu, shiweiliu@fas.harvard.edu',
                  '#experiment_description:':'This experiment set contains integrated RNA- and DNA-MERFISH of the mouse MOp region and a portion of adjacent regions',
                  '#table_description:':'This table contains RNA spot coordinates from the mouse MOp region and a portion of adjacent regions',
                  '#Software_Title:':'Merlin',
                  '#Software_Type:':'SpotLoc + Decoding (by Merlin)',
                  '#Software_Authors:':'George Emanuel, Stephen Eichhorn, Leonardo Sepulveda (Merlin)',
                  '#Software_Description:':'Merlin for RNA-MERFISH decoding',
                  '#Software_Repository:':'https://github.com/emanuega/MERlin',
                  '#Software_PreferredCitationID:':'doi:10.5281/zenodo.3758540',
                  #'#additional_tables:':'4dn_FOF-CT_rna, 4dn_FOF-CT_quality, 4dn_FOF-CT_bio, 4dn_FOF-CT_demultiplexing, 4dn_FOF-CT_trace, 4dn_FOF-CT_cell',
                  '#additional_tables:':'4dn_FOF-CT_core, 4dn_FOF-CT_cell',
                  '##genome_assembly=':'GRCm38(mm10)',
                  '##Gene_ID_type=':'',
                  '##XYZ_unit=':'micron',
                  '##columns=':'(Spot_ID, X, Y, Z, RNA_name, RNA_ID, x_global, y_global, experiment_ID, sample_ID,...)',
                  '#Spot_ID:':'unique RNA spot identifier for the datatable',
                  '#X:':'X pixel coordinate by Merlin',
                  '#Y:':'Y pixel coordinate by Merlin',
                  '#Z:':'pseudo Z pixel coordinate by Merlin',
                  '#RNA_name:':'official name of the Gene the targeted RNA is transcribed from',
                  'Gene_ID:':'official ID for the Gene encoding for the targeted RNA transcript.',
                  '#RNA_ID:':'official ID for the targeted RNA',
                  '#x_global:':'x coordinate converted to match the cell_center_x_coordinate in 4dn_FOF-CT_cell',
                  '#y_global:':'y coordinate converted to match the cell_center_y_coordinate in 4dn_FOF-CT_cell',
                  '#z_global:':'z coordinate converted to match the cell_center_z_coordinate in 4dn_FOF-CT_cell',
                  '#FOV:':'FOV ID across datatables for the same replicate',
                  #'#CellID_byFOV':'specific cell identifier relative to the FOV',
                  #'#Cell_ID:':'Cell_ID has not assigned at this step before RNA spot partitioning',
                 }


In [10]:
# convert header lines to dataframe
header_lines = [k+v for k,v in header_dict_v0.items()]
header_lines_df = pd.DataFrame(header_lines)
header_lines_df

,0
0,##FOF-CT_version=v0.1
1,##Table_namespace=4dn_FOF-CT_RNA
2,#lab_name:Xiaowei Zhuang lab
3,"#experimenter_name:Pu Zheng, Shiwei Liu"
4,"#experimenter_contact:pu_zheng@g.harvard.edu, ..."
5,#experiment_description:This experiment set co...
6,#table_description:This table contains RNA spo...
7,#Software_Title:Merlin
8,#Software_Type:SpotLoc + Decoding (by Merlin)
9,"#Software_Authors:George Emanuel, Stephen Eich..."


# Load and convert anndata

## define parameters for experiment 

In [11]:
# define name and other metadata info in correct order for each experiment below
#exp_name_list = ['20220316exp', '20220402exp', '20220419exp', '20220713exp']
RNA_experiment_ID_list = ['20220304','20220329','20220415','20220418']
#DNA_experiment_ID_list = ['20220316','20220402','20220419','20220713']
sample_ID_list = ['C57BL/6_M_1_MOp_1','C57BL/6_M_2_MOp_1','C57BL/6_M_3_MOp_1','C57BL/6_M_3_MOp_2']

exp_sample_type = 'WT' # to further distinguish the output exp name in case the date is the same
# naming for 4DN alias
Bio_num_list = [1,2,3,3]
Tech_num_list = [1,1,1,2]
table_type = curr_table_name.split('4dn_FOF-CT_')[1]

# output folder to save the converted df
output_folder =r'F:\4DN_deposit\new_version'

In [12]:
# required cols from the data to be processed
required_cols = ['x', 'y', 'global_x','global_y','global_z','fov', 'barcode_id',
                 #'cell_index',
                ]

## load raw data from the merlin output

In [13]:
import anndata

# raw count from merlin
# raw count can also be prepared if you have merged adata for that for example
data_folders= [r'\\10.245.74.158\Chromatin_NAS_8\MERFISH\v2\20220304-storm6_M1', 
               r'\\10.245.74.158\Chromatin_NAS_8\MERFISH\v2\20220329-storm6_M1', 
               r'\\10.245.74.158\Chromatin_NAS_8\MERFISH\v2\20220415-storm65', 
               r'\\10.245.74.158\Chromatin_NAS_8\MERFISH\v2\20220418-storm6',
              ]


## process

In [14]:

for _data_ind, _data_folder in enumerate(data_folders):
    
    experiment_name_key= RNA_experiment_ID_list[_data_ind] + '_' + exp_sample_type
    #data_4dn_savename = os.path.join(output_folder, f'{experiment_name_key}_RNA_table.csv')
    Bio_num = Bio_num_list[_data_ind]
    Tech_num = Tech_num_list[_data_ind]
    data_4dn_savename = os.path.join(output_folder, f'fileproc_genome_wide_brain_ct_B{Bio_num}_T{Tech_num}_{table_type}_f1.csv')
    
    if os.path.exists(data_4dn_savename):
        print (f'Output already exists for {experiment_name_key}. Skip.')
        
    else:
        # load barcode df
        barcode_df_fname = os.path.join(_data_folder, 'ExportBarcodes', 'barcodes.csv')
        barcode_df = pd.read_csv(barcode_df_fname)
        barcode_df ['RNA_experiment_ID'] = RNA_experiment_ID_list[_data_ind]
        # load codebook df
        codebook_fname = os.path.join(_data_folder, 'codebook_0_M1_codebook.csv')
        codebook_df = pd.read_csv(codebook_fname)

        # split by fov for mp
        barcode_df_byFOV = barcode_df.groupby('fov')   
        barcode_df_byFOV_list =[barcode_df_byFOV.get_group(x) for x in barcode_df_byFOV.groups]
        
        ## other mp args
        print(f'Start processing.')
        codebook_df_list = [codebook_df] * len(barcode_df_byFOV_list)
        required_cols_list = [required_cols] * len(barcode_df_byFOV_list)
        gene_id_map_dict_list = [gene_id_map_dict] * len(barcode_df_byFOV_list)
        
        df_4dn_byFOV_list = parallel_task(format_4dn_dataframe_RNA, zip(barcode_df_byFOV_list,
                                                                        codebook_df_list,
                                                                        required_cols_list,
                                                                        gene_id_map_dict_list,
                                                               ))
        
        df_4dn = pd.concat(df_4dn_byFOV_list)

        df_4dn['RNA_experiment_ID'] = RNA_experiment_ID_list[_data_ind]
        df_4dn['Sample_ID'] = sample_ID_list[_data_ind]

        # modify column names and add 4dn header lines
        header_lines_df_fill = header_lines_df.reindex(columns = list(np.arange(0,len(df_4dn.columns))))
        df_4dn_new = pd.DataFrame(np.vstack([df_4dn.columns,df_4dn]))
        df_4dn_clean = pd.concat([header_lines_df_fill,df_4dn_new])

        df_4dn_clean.to_csv(data_4dn_savename, header=None, index=False)
        print(f'Result saved.')


Start processing.
Write in the function to multiprocess as a temp file.
Complete multiprocess; remove the temp file for the function.
00:03:18.03
Result saved.
Start processing.
Write in the function to multiprocess as a temp file.
Complete multiprocess; remove the temp file for the function.
00:02:38.23
Result saved.
Start processing.
Write in the function to multiprocess as a temp file.
Complete multiprocess; remove the temp file for the function.
00:02:09.07
Result saved.
Start processing.
Write in the function to multiprocess as a temp file.
Complete multiprocess; remove the temp file for the function.
00:01:33.76
Result saved.


In [16]:
len(barcode_df_byFOV_list)

200

In [24]:
df_inspect = df_4dn_clean[30:]
df_inspect

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
1,0,597.3963,1400.1575,0.0,Adamts4,ENSMUSG00000006403,ENSMUST00000111314,0,-1594.5812,903.16705,0.0,7,20220418,C57BL/6_M_3_MOp_2
2,1,1186.6904,1705.8329,0.0,Adamts4,ENSMUSG00000006403,ENSMUST00000111314,0,-1530.9374,936.18,0.0,7,20220418,C57BL/6_M_3_MOp_2
3,2,342.90253,278.8285,0.0,Aqp4,ENSMUSG00000024411,ENSMUST00000079081,0,-1622.0665,782.0635,0.0,12,20220418,C57BL/6_M_3_MOp_2
4,3,1020.25,576.5868,0.0,Aqp4,ENSMUSG00000024411,ENSMUST00000079081,0,-1548.913,814.2214,0.0,12,20220418,C57BL/6_M_3_MOp_2
5,4,1042.3392,1297.265,0.0,Aqp4,ENSMUSG00000024411,ENSMUST00000079081,0,-1546.5273,892.0546,0.0,12,20220418,C57BL/6_M_3_MOp_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6410506,6410505,1005.1019,1219.1671,10.0,Ramp1,ENSMUSG00000034353,ENSMUST00000097648,199,784.651,-418.97998,10.0,176,20220418,C57BL/6_M_3_MOp_2
6410507,6410506,1273.211,1452.0,10.0,Wipf3,ENSMUSG00000086040,ENSMUST00000163746,199,813.60675,-393.834,10.0,239,20220418,C57BL/6_M_3_MOp_2
6410508,6410507,1004.35626,1258.3467,11.0,Ano4,ENSMUSG00000035189,ENSMUST00000182341,199,784.57043,-414.74857,11.0,11,20220418,C57BL/6_M_3_MOp_2
6410509,6410508,969.15967,1606.3683,11.0,Lypd1,ENSMUSG00000026344,ENSMUST00000159417,199,780.7692,-377.16223,11.0,117,20220418,C57BL/6_M_3_MOp_2


In [25]:
np.unique(df_inspect[7])

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
       168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180,
       181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193,
       194, 195, 196, 197, 198, 199], dtype=objec